In [ ]:
# %rm -rf phase2_module
# !git clone https://github.com/LgDNet/phase2_module.git
# !cd phase2_module && make copy

In [ ]:
pwd

In [1]:
cd ..

c:\Users\승범 pc\Desktop\공부\딥러닝\엘지에이머스


In [2]:
from phase2_module.src.utils.dataset import Data
from phase2_module.src.models.classification.models import Model
from phase2_module.src.base import BasePiepline
# from phase2_module.src.hyper_parameters.params_optimization import xgboost_opt, set_params_optimization_data
from phase2_module.src.utils.set_seed import seed_everything
from phase2_module.src.utils.top_score_instance import check_the_score

from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
from phase2_module.src import preprocess
seed_everything()  # NOTE: default 42
_model = Model()

In [3]:
import pandas as pd
import numpy as np
import re, pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
def set_up(mode= None):
    pd.set_option('display.max_columns', mode)
set_up()

# Preprocessing

In [4]:
basic = preprocess.Basic
country = preprocess.Country
customer = preprocess.Customer
inquiry = preprocess.Inquiry
product_category = preprocess.ProductCategory
convert_ratio = preprocess.Convert_ratio

# Main

In [5]:
pkls = pkl_manager.map(pkl_manager.loads, pkl_manager.metadata_directory)

runtime_instance = {
    "Train": Data.train,
    "Test": Data.test,
    "Instances":[
        {"instance": basic, "method": ['drop_duplicated'], "params": False},
        {"instance": country, "method": ['apply'], "params": pkls.get("country")},
        {"instance": inquiry, "method": ["apply"], "params": False},
        {"instance": product_category, "method": ["product_categories", 'fill_odds'], "params": {**pkls.get("product_category"), **pkls.get("product_subcategory")}},
        {"instance": customer, "method": ["apply"], "params": pkls.get("customer")},
        {"instance": convert_ratio, "method": ["lead_owner","customer_idx",'lead_owner_customer_idx_means'], "params": {**pkls.get("lead_owner"), **pkls.get("customer_idx")}},
        {"instance": basic, "method": ['cusotmer_idx_categorization'], "params": False},
    ]
}

## preprocessing

In [253]:
df_train = preprocess.runner('Train',**runtime_instance)
df_test = preprocess.runner('Test',**runtime_instance)

In [254]:
# russia & csi 변환
cond = df_train['continent']=='russia & cis'
df_train.loc[cond,'continent'] = 'asia & pacific'
# df_test에 러시아가 없음.

In [255]:
# cm 제거
cond2 = df_train[df_train['business_unit'] == 'CM']
df_train.drop(cond2.index,axis = 0, inplace = True)
df_train.reset_index(drop = True,inplace = True)

In [256]:
cond = df_train['historical_existing_cnt']> 0 
df_train.loc[cond,"prev"] = 1
df_train['prev'] = df_train['prev'].fillna(0)

cond = df_test['historical_existing_cnt']> 0 
df_test.loc[cond,"prev"] = 1
df_test['prev'] = df_test['prev'].fillna(0)

* 기존에 train에 있는 애들이면, train의 데이터로 채워야하지 않아? customer_idx가?


## encoding

In [257]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
class Encode:
    def __init__(self):
        pass
    
    def label_encoder(self,df,columns):
        if not encoding_zip['label']:
            for col in columns: # train
                encoder = LabelEncoder()
                try:
                    df.loc[:,col] = encoder.fit_transform(df[col])
                except Exception as e:
                    print(e)
                    print(col)
                # 인코더 저장
                encoding_zip['label'].append(encoder)
        else: # test
            for idx, col in enumerate(columns):
                encoder = encoding_zip['label'][idx]
                
                for label in np.sort(df[col].unique()):
                    if label not in encoder.classes_:
                        encoder.classes_ = np.append(encoder.classes_,label)
                df.loc[:,col] = encoder.transform(df[col])
        return df
    
    def convert_dataframe(self,df, onehot_arr, category_list, col_):
        col_list = np.concatenate([i.flatten() for i in category_list])    
        onehot_df = pd.DataFrame(onehot_arr, columns=[f"OneHot_{col_name}"\
                                          for col_name in col_list])
        df = pd.concat([df,onehot_df],axis = 1)
        df.drop(col_,axis = 1, inplace = True)
        return df
    
    def onehot_encoder(self,df,columns):
        if not encoding_zip['onehot']:
            encoder = OneHotEncoder(sparse=False)
            try:
                onehot_arr = encoder.fit_transform(df[columns])
                df = self.convert_dataframe(df,onehot_arr,encoder.categories_,columns)
            except Exception as e:
                print(e, col)
            # 인코더 저장
            encoding_zip['onehot'].append(encoder)
        else:
            encoder = encoding_zip['onehot'][0]
            onehot_arr = encoder.transform(df[columns])
            df = self.convert_dataframe(df,onehot_arr,encoder.categories_,columns)
        return df

In [258]:
encode = Encode()
encoding_zip = {"label":[], "onehot":[]}

In [259]:
exisiting_columns = [
                     'business_area','business_subarea','business_unit',
                        'enterprise',
#                         'response_corporate',"customer_country", # country
#                         'customer_type','customer_job','customer_position', #customer
#                         'inquiry_type','expected_timeline', # inquiry
#                         'product_subcategory','product_category','product_modelname' # product_category
                       ] 
preprocess_columns = [
                     'country','continent', # country
                     'customer_type2','job_function','seniority_level', # customer
                     'new_expected_timeline', 'new_inquiry_type',# inquiry
                     'category_2','category_3','category_1', # product_category
#                      'test2'
                     ]

label_encode_columns = exisiting_columns + preprocess_columns

In [260]:
# 결측치 채우기
def fillna(df):
    for col in df.columns:
        if df[col].isna().sum():
            if df[col].dtypes =='O':
                df[col].fillna('Space', inplace = True)
            else:
                df[col].fillna(0,inplace = True)
    return df
df_train = fillna(df_train)
df_test = fillna(df_test)

In [ ]:
# # one-hot encoding
# onehot_encode_columns = ['category_1']
# df_train = encode.onehot_encoder(df_train,onehot_encode_columns)
# df_test = encode.onehot_encoder(df_test,onehot_encode_columns)

In [261]:
# label encoding
df_train = encode.label_encoder(df_train,label_encode_columns)
df_test = encode.label_encoder(df_test,label_encode_columns)

# Modeling

## drop column
모델링 전이 column을 drop하면서 다루기가 쉽다고 판단하였음

In [263]:
drop_col = [
            'id_strategic_ver','it_strategic_ver',
            'customer_country.1',
    
            'response_corporate',"customer_country", # country
            'customer_type','customer_job','customer_position', # customer
            'inquiry_type', 'expected_timeline', #inquiry
            'product_subcategory','product_category','product_modelname', # product_category
#             'cate_is_nan', # product_category
    
#             'com_reg_ver_win_rate','idit_strategic_ver','ver_cus',
#             'ver_pro','ver_win_rate_x','ver_win_ratio_per_bu',
            'customer_idx',
            'customer_idx_converted_ratio', # ratio
            'idit_strategic_ver','ver_pro', # test
            'business_unit','historical_existing_cnt',
            'lead_owner', 'lead_owner_converted_ratio', 'lead_owner_customer_idx_mean', # lead_owner
            ]
df_train.drop(drop_col,axis = 1, inplace = True)
df_test.drop(drop_col,axis = 1, inplace = True)

## Scaler

In [264]:
import numpy as np
# log 변환
df_train['lead_desc_length'] = np.log1p(df_train['lead_desc_length'])
df_test['lead_desc_length'] = np.log1p(df_test['lead_desc_length'])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']] = scaler.fit_transform(df_train[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']])
df_test[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']] = scaler.transform(df_test[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']])
# df_test['lead_desc_length'] = scale.transform(df_test['lead_desc_length'].values.reshape(-1, 1))

## train

In [265]:
dict_ = {'criterion': 'log_loss', 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 2, 'random_state': 42}
model = _model.decision_tree
model.set_params(**dict_)

DecisionTreeClassifier(criterion='log_loss', max_depth=20, min_samples_leaf=2,
                       min_samples_split=7, random_state=42)

In [146]:
model = _model.xgboost
model.set_params(random_state =42)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [266]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']

In [267]:
from sklearn.model_selection import StratifiedKFold
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[],'roc_auc_score':[]}
train_result = {"f1":[], "precision":[], "recall":[],'roc_auc_score':[]}
models = [] # 모델을 저장할 리스트

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    # 모델 훈련
    model.fit(x_train, y_train)
    models.append(model) # 훈련된 모델을 리스트에 추가
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)

    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)

    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)

print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

5it [00:01,  4.64it/s]

----[K-Fold Train Score]-----
f1 score : 0.8834 / STD: (+/- 0.0032)
precision score : 0.8324 / STD: (+/- 0.0072)
recall score : 0.9411 / STD: (+/- 0.0047)
roc_auc_score score : 0.9631 / STD: (+/- 0.0022)
----[K-Fold Validation Score]-----
f1 score : 0.7288 / STD: (+/- 0.0063)
precision score : 0.6903 / STD: (+/- 0.0041)
recall score : 0.7721 / STD: (+/- 0.0153)
roc_auc_score score : 0.8722 / STD: (+/- 0.0076)


In [269]:
feature_importances = model.feature_importances_
feature_names = x_train.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

prev: 0.34949255167475185
customer_idx_group: 0.15133137566914112
lead_desc_length: 0.08074142066698692
customer_type2: 0.06292536345408146
business_subarea: 0.06213368764118546
country: 0.04392539278013874
job_function: 0.0384062172016642
com_reg_ver_win_rate: 0.034791226622388025
category_3: 0.024145000758120004
seniority_level: 0.021683949456995193
enterprise: 0.01820059995169897
continent: 0.01672830735879867
category_1: 0.016648924083319975
bant_submit: 0.01588730708436977
category_2: 0.014415585381649939
new_expected_timeline: 0.012754879643624586
ver_win_ratio_per_bu: 0.009366082745080526
customer_interest: 0.008169250310148684
business_area: 0.006472820210834474
ver_win_rate_x: 0.004950563307317844
new_inquiry_type: 0.004908695273646238
ver_cus: 0.0019207987240574564


In [ ]:
lead_owner_converted_ratio: 0.3015696383760019
historical_existing_cnt: 0.260322747670651
customer_idx_group: 0.11237190087589338
lead_owner_customer_idx_mean: 0.051446014673893745
lead_owner: 0.044515661338854234
lead_desc_length: 0.03767199295523489
customer_type2: 0.03189911091363563
com_reg_ver_win_rate: 0.02630451208522597
business_subarea: 0.018585638860141668
job_function: 0.017359754715912355
category_3: 0.014769937160322238
country: 0.013179620346437115
seniority_level: 0.01196550300828415
continent: 0.00834325717171033
new_expected_timeline: 0.00733746910012691
category_2: 0.006888513212260396
customer_interest: 0.006641276293499343
business_unit: 0.006226575085287268
bant_submit: 0.005670569449475212
enterprise: 0.004800146013144287
business_area: 0.004152473335310216
ver_win_ratio_per_bu: 0.0035102704885596196
ver_win_rate_x: 0.0017318604280630275
category_1: 0.0016282339962348062
new_inquiry_type: 0.0006380581678085043
ver_cus: 0.0004692642780317513

## test & submission

In [270]:
test_drop = df_test.drop(['id','is_converted'],axis = 1)

In [ ]:
# pred = model.predict(test_drop)
# pred = np.where(pred == 0, False, True)

In [271]:
final_predictions = np.array([model.predict(test_drop) for model in models])
final_prediction = np.round(final_predictions.mean(axis=0))
pred = np.where(final_prediction == 0, False, True)

In [272]:
sum(pred)

859

In [273]:
pred1 = pred  #pred1 1000 | pred2 919 | pred3 820

In [274]:
su = 0
for i,j in zip(pred2, pred1):
    if i != j:
        su +=1
su

1240

In [275]:
len(df_test)

5271

In [ ]:
pred1_ = pred1.reshape(-1,1)
pred2_ = pred2.reshape(-1,1)
pred3_ = pred3.reshape(-1,1)
combine = np.hstack((pred1_, pred2_, pred3_))
combine_ = np.round(combine.mean(axis=1))

In [ ]:
pred = np.where(combine_ == 0, False, True)

In [ ]:
pred

In [277]:
df_test['is_converted'] = pred
df_test.to_csv("submission.csv",index = False)